# API Usage Tutorial for AG News Classification

## Overview

This notebook demonstrates comprehensive API usage patterns following methodologies from:
- Fielding (2000): "Architectural Styles and the Design of Network-based Software Architectures"
- Google (2015): "gRPC: A high performance, open source universal RPC framework"
- Facebook (2015): "GraphQL: A Query Language for Your API"

### Tutorial Objectives
1. Connect to REST API endpoints
2. Implement gRPC client communication
3. Execute GraphQL queries and mutations
4. Handle authentication and rate limiting
5. Implement batch processing and streaming
6. Monitor API performance and health

Author: Võ Hải Dũng  
Email: vohaidung.work@gmail.com  
Date: 2025

## 1. Environment Setup

In [ ]:
# Standard library imports
import sys
import os
import json
import asyncio
import time
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any, Union
from dataclasses import dataclass, asdict
import warnings

# HTTP and API imports
import requests
import httpx
import websocket
import grpc
from gql import gql, Client
from gql.transport.httpx import HTTPXTransport

# Data manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

# Project imports
PROJECT_ROOT = Path("../..").resolve()
sys.path.insert(0, str(PROJECT_ROOT))

from src.api.rest.schemas.request_schemas import (
    ClassificationRequest,
    BatchClassificationRequest,
    TrainingRequest
)
from src.api.rest.schemas.response_schemas import (
    ClassificationResponse,
    BatchClassificationResponse,
    ModelInfoResponse
)
from src.api.grpc.compiled import classification_pb2, classification_pb2_grpc
from src.api.grpc.compiled import model_management_pb2, model_management_pb2_grpc
from src.utils.api_utils import (
    create_api_client,
    handle_api_error,
    retry_with_backoff
)
from src.utils.logging_config import setup_logging
from configs.config_loader import ConfigLoader
from configs.constants import AG_NEWS_CLASSES

# Setup
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
logger = setup_logging('api_usage_tutorial')

# API Configuration
API_CONFIG = {
    'rest_base_url': 'http://localhost:8000',
    'grpc_host': 'localhost',
    'grpc_port': 50051,
    'graphql_url': 'http://localhost:8000/graphql',
    'api_key': os.getenv('API_KEY', 'demo_api_key'),
    'timeout': 30
}

print("API Usage Tutorial")
print("="*50)
print(f"REST API URL: {API_CONFIG['rest_base_url']}")
print(f"gRPC Server: {API_CONFIG['grpc_host']}:{API_CONFIG['grpc_port']}")
print(f"GraphQL Endpoint: {API_CONFIG['graphql_url']}")

## 2. REST API Setup and Authentication

In [ ]:
class RESTAPIClient:
    """
    REST API client for AG News classification service.
    
    Following REST principles from:
        Richardson & Ruby (2007): "RESTful Web Services"
    """
    
    def __init__(self, base_url: str, api_key: str, timeout: int = 30):
        self.base_url = base_url
        self.api_key = api_key
        self.timeout = timeout
        self.session = self._create_session()
    
    def _create_session(self) -> requests.Session:
        """Create configured HTTP session."""
        session = requests.Session()
        session.headers.update({
            'Authorization': f'Bearer {self.api_key}',
            'Content-Type': 'application/json',
            'Accept': 'application/json',
            'User-Agent': 'AGNews-Tutorial/1.0'
        })
        return session
    
    def health_check(self) -> Dict[str, Any]:
        """Check API health status."""
        response = self.session.get(
            f"{self.base_url}/health",
            timeout=self.timeout
        )
        response.raise_for_status()
        return response.json()
    
    def classify_text(self, text: str, model_id: Optional[str] = None) -> Dict[str, Any]:
        """Classify single text."""
        payload = {
            'text': text,
            'model_id': model_id or 'default'
        }
        
        response = self.session.post(
            f"{self.base_url}/api/v1/classify",
            json=payload,
            timeout=self.timeout
        )
        response.raise_for_status()
        return response.json()
    
    def batch_classify(self, texts: List[str], model_id: Optional[str] = None) -> Dict[str, Any]:
        """Classify multiple texts in batch."""
        payload = {
            'texts': texts,
            'model_id': model_id or 'default',
            'return_probabilities': True
        }
        
        response = self.session.post(
            f"{self.base_url}/api/v1/classify/batch",
            json=payload,
            timeout=self.timeout
        )
        response.raise_for_status()
        return response.json()
    
    def get_models(self) -> List[Dict[str, Any]]:
        """Get available models."""
        response = self.session.get(
            f"{self.base_url}/api/v1/models",
            timeout=self.timeout
        )
        response.raise_for_status()
        return response.json()['models']
    
    def get_metrics(self, model_id: str) -> Dict[str, Any]:
        """Get model metrics."""
        response = self.session.get(
            f"{self.base_url}/api/v1/metrics/{model_id}",
            timeout=self.timeout
        )
        response.raise_for_status()
        return response.json()


# Initialize REST client
rest_client = RESTAPIClient(
    base_url=API_CONFIG['rest_base_url'],
    api_key=API_CONFIG['api_key'],
    timeout=API_CONFIG['timeout']
)

# Test health check
print("Testing REST API Connection:")
print("="*50)

try:
    health_status = rest_client.health_check()
    print(f"API Status: {health_status.get('status', 'unknown')}")
    print(f"Version: {health_status.get('version', 'unknown')}")
    print(f"Uptime: {health_status.get('uptime', 'unknown')}")
except Exception as e:
    print(f"Error connecting to REST API: {e}")
    print("Please ensure the API server is running.")

## 3. REST API Usage Examples

In [ ]:
# Sample texts for testing
sample_texts = [
    "Apple reported record quarterly revenue driven by strong iPhone sales.",
    "Scientists discover new exoplanet using advanced telescope technology.",
    "The Lakers defeated the Celtics in overtime thriller last night.",
    "UN Security Council meets to discuss ongoing international crisis."
]

print("REST API Classification Examples:")
print("="*50)

# Single text classification
print("\n1. Single Text Classification:")
print("-"*40)

try:
    result = rest_client.classify_text(sample_texts[0])
    print(f"Text: {sample_texts[0][:50]}...")
    print(f"Predicted Category: {result['category']}")
    print(f"Confidence: {result['confidence']:.3f}")
    print(f"Processing Time: {result.get('processing_time_ms', 'N/A')} ms")
except Exception as e:
    print(f"Error: {e}")

# Batch classification
print("\n2. Batch Classification:")
print("-"*40)

try:
    batch_results = rest_client.batch_classify(sample_texts)
    
    for i, result in enumerate(batch_results['predictions']):
        print(f"\nText {i+1}: {sample_texts[i][:50]}...")
        print(f"  Category: {result['category']}")
        print(f"  Confidence: {result['confidence']:.3f}")
        if 'probabilities' in result:
            print(f"  Probabilities:")
            for cls, prob in result['probabilities'].items():
                print(f"    {cls}: {prob:.3f}")
except Exception as e:
    print(f"Error: {e}")

# Get available models
print("\n3. Available Models:")
print("-"*40)

try:
    models = rest_client.get_models()
    
    for model in models:
        print(f"\nModel ID: {model['id']}")
        print(f"  Name: {model['name']}")
        print(f"  Type: {model['type']}")
        print(f"  Status: {model['status']}")
        print(f"  Accuracy: {model.get('accuracy', 'N/A')}")
except Exception as e:
    print(f"Error: {e}")

## 4. gRPC API Setup

In [ ]:
class GRPCClient:
    """
    gRPC client for AG News classification service.
    
    Following gRPC best practices from:
        Google Cloud (2023): "gRPC Best Practices"
    """
    
    def __init__(self, host: str, port: int, api_key: str):
        self.host = host
        self.port = port
        self.api_key = api_key
        self.channel = None
        self.classification_stub = None
        self.model_stub = None
        self._connect()
    
    def _connect(self):
        """Establish gRPC connection."""
        # Create channel with interceptors
        self.channel = grpc.insecure_channel(f'{self.host}:{self.port}')
        
        # Add metadata for authentication
        metadata = [('authorization', f'Bearer {self.api_key}')]
        
        # Create stubs
        self.classification_stub = classification_pb2_grpc.ClassificationServiceStub(self.channel)
        self.model_stub = model_management_pb2_grpc.ModelManagementServiceStub(self.channel)
    
    def classify(self, text: str, model_id: str = 'default') -> classification_pb2.ClassificationResponse:
        """Classify text using gRPC."""
        request = classification_pb2.ClassificationRequest(
            text=text,
            model_id=model_id
        )
        
        metadata = [('authorization', f'Bearer {self.api_key}')]
        response = self.classification_stub.Classify(request, metadata=metadata)
        return response
    
    def stream_classify(self, texts: List[str]) -> Any:
        """Stream classification requests."""
        def request_generator():
            for text in texts:
                yield classification_pb2.ClassificationRequest(
                    text=text,
                    model_id='default'
                )
        
        metadata = [('authorization', f'Bearer {self.api_key}')]
        responses = self.classification_stub.StreamClassify(
            request_generator(),
            metadata=metadata
        )
        
        return responses
    
    def get_model_info(self, model_id: str) -> model_management_pb2.ModelInfo:
        """Get model information."""
        request = model_management_pb2.GetModelRequest(model_id=model_id)
        metadata = [('authorization', f'Bearer {self.api_key}')]
        response = self.model_stub.GetModel(request, metadata=metadata)
        return response
    
    def close(self):
        """Close gRPC channel."""
        if self.channel:
            self.channel.close()


# Initialize gRPC client
print("gRPC API Setup:")
print("="*50)

try:
    grpc_client = GRPCClient(
        host=API_CONFIG['grpc_host'],
        port=API_CONFIG['grpc_port'],
        api_key=API_CONFIG['api_key']
    )
    print(f"Connected to gRPC server: {API_CONFIG['grpc_host']}:{API_CONFIG['grpc_port']}")
    
    # Test classification
    print("\nTesting gRPC Classification:")
    response = grpc_client.classify(sample_texts[0])
    print(f"Text: {sample_texts[0][:50]}...")
    print(f"Category: {response.category}")
    print(f"Confidence: {response.confidence:.3f}")
    
    # Test streaming
    print("\nTesting Streaming Classification:")
    stream_responses = grpc_client.stream_classify(sample_texts[:2])
    
    for i, response in enumerate(stream_responses):
        print(f"  Result {i+1}: {response.category} (confidence: {response.confidence:.3f})")
    
except Exception as e:
    print(f"Error connecting to gRPC server: {e}")
    print("Please ensure the gRPC server is running.")
    grpc_client = None

## 5. GraphQL API Setup

In [ ]:
class GraphQLClient:
    """
    GraphQL client for AG News classification service.
    
    Following GraphQL best practices from:
        Porcello & Banks (2018): "Learning GraphQL"
    """
    
    def __init__(self, url: str, api_key: str):
        self.url = url
        self.api_key = api_key
        self.client = self._create_client()
    
    def _create_client(self) -> Client:
        """Create GraphQL client."""
        transport = HTTPXTransport(
            url=self.url,
            headers={'Authorization': f'Bearer {self.api_key}'}
        )
        return Client(transport=transport, fetch_schema_from_transport=True)
    
    def classify(self, text: str, model_id: str = 'default') -> Dict[str, Any]:
        """Classify text using GraphQL."""
        query = gql("""
            mutation ClassifyText($text: String!, $modelId: String) {
                classify(text: $text, modelId: $modelId) {
                    category
                    confidence
                    probabilities {
                        category
                        probability
                    }
                    processingTime
                }
            }
        """)
        
        variables = {'text': text, 'modelId': model_id}
        result = self.client.execute(query, variable_values=variables)
        return result['classify']
    
    def batch_classify(self, texts: List[str]) -> List[Dict[str, Any]]:
        """Batch classify texts."""
        query = gql("""
            mutation BatchClassify($texts: [String!]!) {
                batchClassify(texts: $texts) {
                    predictions {
                        category
                        confidence
                        index
                    }
                    totalProcessingTime
                }
            }
        """)
        
        variables = {'texts': texts}
        result = self.client.execute(query, variable_values=variables)
        return result['batchClassify']['predictions']
    
    def get_models(self) -> List[Dict[str, Any]]:
        """Get available models."""
        query = gql("""
            query GetModels {
                models {
                    id
                    name
                    type
                    status
                    accuracy
                    lastUpdated
                }
            }
        """)
        
        result = self.client.execute(query)
        return result['models']
    
    def subscribe_to_predictions(self) -> Any:
        """Subscribe to real-time predictions."""
        subscription = gql("""
            subscription OnPrediction {
                predictionMade {
                    text
                    category
                    confidence
                    timestamp
                }
            }
        """)
        
        # This would return an async iterator in production
        return subscription


# Initialize GraphQL client
print("GraphQL API Setup:")
print("="*50)

try:
    graphql_client = GraphQLClient(
        url=API_CONFIG['graphql_url'],
        api_key=API_CONFIG['api_key']
    )
    print(f"Connected to GraphQL endpoint: {API_CONFIG['graphql_url']}")
    
    # Test classification
    print("\nTesting GraphQL Classification:")
    result = graphql_client.classify(sample_texts[1])
    print(f"Text: {sample_texts[1][:50]}...")
    print(f"Category: {result['category']}")
    print(f"Confidence: {result['confidence']:.3f}")
    
    if 'probabilities' in result:
        print("Probabilities:")
        for prob in result['probabilities']:
            print(f"  {prob['category']}: {prob['probability']:.3f}")
    
    # Test batch classification
    print("\nTesting Batch Classification:")
    batch_results = graphql_client.batch_classify(sample_texts[:2])
    
    for result in batch_results:
        print(f"  Index {result['index']}: {result['category']} ({result['confidence']:.3f})")
    
except Exception as e:
    print(f"Error connecting to GraphQL API: {e}")
    print("Please ensure the GraphQL server is running.")
    graphql_client = None

## 6. WebSocket Real-time Communication

In [ ]:
class WebSocketClient:
    """
    WebSocket client for real-time classification.
    
    Following WebSocket protocol from:
        RFC 6455: "The WebSocket Protocol"
    """
    
    def __init__(self, url: str, api_key: str):
        self.url = url
        self.api_key = api_key
        self.ws = None
    
    def connect(self):
        """Establish WebSocket connection."""
        self.ws = websocket.WebSocketApp(
            self.url,
            header={'Authorization': f'Bearer {self.api_key}'},
            on_open=self.on_open,
            on_message=self.on_message,
            on_error=self.on_error,
            on_close=self.on_close
        )
    
    def on_open(self, ws):
        """Handle connection open."""
        print("WebSocket connection established")
    
    def on_message(self, ws, message):
        """Handle incoming message."""
        data = json.loads(message)
        print(f"Received: {data}")
    
    def on_error(self, ws, error):
        """Handle error."""
        print(f"WebSocket error: {error}")
    
    def on_close(self, ws, close_status_code, close_msg):
        """Handle connection close."""
        print("WebSocket connection closed")
    
    def send_for_classification(self, text: str):
        """Send text for classification."""
        if self.ws:
            message = json.dumps({
                'action': 'classify',
                'text': text,
                'timestamp': time.time()
            })
            self.ws.send(message)
    
    def close(self):
        """Close WebSocket connection."""
        if self.ws:
            self.ws.close()


# Demonstrate WebSocket usage
print("WebSocket Real-time Communication:")
print("="*50)

ws_url = f"ws://localhost:8000/ws/classify"

try:
    ws_client = WebSocketClient(ws_url, API_CONFIG['api_key'])
    print(f"Connecting to WebSocket: {ws_url}")
    
    # Note: In production, this would run asynchronously
    print("WebSocket client configured (would connect in production)")
    print("\nWebSocket operations:")
    print("  - Real-time classification")
    print("  - Streaming predictions")
    print("  - Live model updates")
    print("  - Performance monitoring")
    
except Exception as e:
    print(f"Error setting up WebSocket: {e}")

## 7. Error Handling and Retry Logic

In [ ]:
import time
from typing import Callable, Any
import random


class APIErrorHandler:
    """
    Comprehensive error handling for API calls.
    
    Following error handling patterns from:
        Nygard (2007): "Release It! Design and Deploy Production-Ready Software"
    """
    
    @staticmethod
    def exponential_backoff(
        func: Callable,
        max_retries: int = 3,
        base_delay: float = 1.0,
        max_delay: float = 60.0
    ) -> Any:
        """Retry with exponential backoff."""
        for attempt in range(max_retries):
            try:
                return func()
            except Exception as e:
                if attempt == max_retries - 1:
                    raise
                
                # Calculate delay with jitter
                delay = min(base_delay * (2 ** attempt) + random.uniform(0, 1), max_delay)
                print(f"Attempt {attempt + 1} failed: {e}")
                print(f"Retrying in {delay:.2f} seconds...")
                time.sleep(delay)
    
    @staticmethod
    def circuit_breaker(
        func: Callable,
        failure_threshold: int = 5,
        recovery_timeout: float = 60.0
    ) -> Any:
        """Implement circuit breaker pattern."""
        # Simplified circuit breaker implementation
        failures = 0
        last_failure_time = None
        
        if failures >= failure_threshold:
            if time.time() - last_failure_time < recovery_timeout:
                raise Exception("Circuit breaker is open")
            else:
                failures = 0  # Reset after recovery timeout
        
        try:
            result = func()
            failures = 0  # Reset on success
            return result
        except Exception as e:
            failures += 1
            last_failure_time = time.time()
            raise


# Test error handling
print("Error Handling Examples:")
print("="*50)

# Simulate API call with potential failures
def unreliable_api_call():
    """Simulate unreliable API call."""
    if random.random() < 0.7:  # 70% chance of failure
        raise ConnectionError("API temporarily unavailable")
    return {'status': 'success', 'data': 'Classification result'}

# Test exponential backoff
print("\n1. Testing Exponential Backoff:")
print("-"*40)

error_handler = APIErrorHandler()

try:
    result = error_handler.exponential_backoff(
        unreliable_api_call,
        max_retries=3,
        base_delay=1.0
    )
    print(f"Success: {result}")
except Exception as e:
    print(f"Failed after retries: {e}")

# Rate limiting handler
class RateLimiter:
    """
    Handle API rate limiting.
    
    Following rate limiting strategies from:
        Cloud API Best Practices
    """
    
    def __init__(self, max_requests: int = 100, window_seconds: int = 60):
        self.max_requests = max_requests
        self.window_seconds = window_seconds
        self.requests = []
    
    def can_make_request(self) -> bool:
        """Check if request can be made."""
        now = time.time()
        
        # Remove old requests outside window
        self.requests = [
            req_time for req_time in self.requests
            if now - req_time < self.window_seconds
        ]
        
        if len(self.requests) < self.max_requests:
            self.requests.append(now)
            return True
        return False
    
    def wait_time(self) -> float:
        """Calculate wait time until next request."""
        if len(self.requests) < self.max_requests:
            return 0
        
        oldest_request = min(self.requests)
        wait = self.window_seconds - (time.time() - oldest_request)
        return max(0, wait)


# Test rate limiting
print("\n2. Testing Rate Limiting:")
print("-"*40)

rate_limiter = RateLimiter(max_requests=5, window_seconds=10)

for i in range(8):
    if rate_limiter.can_make_request():
        print(f"Request {i+1}: Allowed")
    else:
        wait_time = rate_limiter.wait_time()
        print(f"Request {i+1}: Rate limited (wait {wait_time:.1f}s)")

## 8. Performance Testing

In [ ]:
class APIPerformanceTester:
    """
    Performance testing for API endpoints.
    
    Following performance testing practices from:
        Molyneaux (2009): "The Art of Application Performance Testing"
    """
    
    def __init__(self, client: RESTAPIClient):
        self.client = client
        self.results = []
    
    def latency_test(
        self,
        text: str,
        iterations: int = 10
    ) -> Dict[str, float]:
        """Test API latency."""
        latencies = []
        
        for _ in range(iterations):
            start_time = time.perf_counter()
            try:
                _ = self.client.classify_text(text)
                latency = (time.perf_counter() - start_time) * 1000  # Convert to ms
                latencies.append(latency)
            except Exception as e:
                print(f"Request failed: {e}")
        
        if latencies:
            return {
                'min': min(latencies),
                'max': max(latencies),
                'mean': np.mean(latencies),
                'median': np.median(latencies),
                'p95': np.percentile(latencies, 95),
                'p99': np.percentile(latencies, 99)
            }
        return {}
    
    def throughput_test(
        self,
        texts: List[str],
        duration_seconds: int = 10
    ) -> Dict[str, float]:
        """Test API throughput."""
        start_time = time.time()
        requests_completed = 0
        errors = 0
        
        while time.time() - start_time < duration_seconds:
            try:
                text = random.choice(texts)
                _ = self.client.classify_text(text)
                requests_completed += 1
            except Exception:
                errors += 1
        
        elapsed = time.time() - start_time
        
        return {
            'total_requests': requests_completed + errors,
            'successful_requests': requests_completed,
            'failed_requests': errors,
            'requests_per_second': requests_completed / elapsed,
            'success_rate': requests_completed / (requests_completed + errors) if (requests_completed + errors) > 0 else 0
        }
    
    def concurrent_test(
        self,
        texts: List[str],
        concurrent_requests: int = 5
    ) -> Dict[str, Any]:
        """Test concurrent request handling."""
        import concurrent.futures
        
        def make_request(text):
            start = time.perf_counter()
            try:
                result = self.client.classify_text(text)
                return {
                    'success': True,
                    'latency': (time.perf_counter() - start) * 1000,
                    'result': result
                }
            except Exception as e:
                return {
                    'success': False,
                    'latency': (time.perf_counter() - start) * 1000,
                    'error': str(e)
                }
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=concurrent_requests) as executor:
            futures = [executor.submit(make_request, text) for text in texts[:concurrent_requests]]
            results = [future.result() for future in concurrent.futures.as_completed(futures)]
        
        successful = [r for r in results if r['success']]
        failed = [r for r in results if not r['success']]
        
        return {
            'total_requests': len(results),
            'successful': len(successful),
            'failed': len(failed),
            'avg_latency': np.mean([r['latency'] for r in results]),
            'max_latency': max([r['latency'] for r in results])
        }


# Run performance tests
print("API Performance Testing:")
print("="*50)

if rest_client:
    perf_tester = APIPerformanceTester(rest_client)
    
    # Latency test
    print("\n1. Latency Test (10 iterations):")
    print("-"*40)
    
    latency_results = perf_tester.latency_test(sample_texts[0], iterations=10)
    
    if latency_results:
        for metric, value in latency_results.items():
            print(f"  {metric:6}: {value:.2f} ms")
    
    # Throughput test (simulated)
    print("\n2. Throughput Test (simulated):")
    print("-"*40)
    
    # Simulated results for demonstration
    throughput_results = {
        'total_requests': 150,
        'successful_requests': 145,
        'failed_requests': 5,
        'requests_per_second': 14.5,
        'success_rate': 0.967
    }
    
    for metric, value in throughput_results.items():
        if 'rate' in metric:
            print(f"  {metric}: {value:.3f}")
        elif 'per_second' in metric:
            print(f"  {metric}: {value:.1f}")
        else:
            print(f"  {metric}: {value}")
    
    # Concurrent test (simulated)
    print("\n3. Concurrent Request Test (simulated):")
    print("-"*40)
    
    concurrent_results = {
        'total_requests': 5,
        'successful': 5,
        'failed': 0,
        'avg_latency': 125.4,
        'max_latency': 187.2
    }
    
    for metric, value in concurrent_results.items():
        if 'latency' in metric:
            print(f"  {metric}: {value:.2f} ms")
        else:
            print(f"  {metric}: {value}")
else:
    print("REST client not available for performance testing.")

## 9. API Monitoring and Health Checks

In [ ]:
class APIMonitor:
    """
    Monitor API health and performance.
    
    Following monitoring practices from:
        Turnbull (2018): "The Art of Monitoring"
    """
    
    def __init__(self, clients: Dict[str, Any]):
        self.clients = clients
        self.metrics = []
    
    def health_check_all(self) -> Dict[str, Dict[str, Any]]:
        """Check health of all API endpoints."""
        results = {}
        
        # REST API health check
        if 'rest' in self.clients and self.clients['rest']:
            try:
                health = self.clients['rest'].health_check()
                results['REST'] = {
                    'status': 'healthy',
                    'details': health
                }
            except Exception as e:
                results['REST'] = {
                    'status': 'unhealthy',
                    'error': str(e)
                }
        
        # gRPC health check
        if 'grpc' in self.clients and self.clients['grpc']:
            try:
                # Simulate gRPC health check
                results['gRPC'] = {
                    'status': 'healthy',
                    'details': {'serving': True}
                }
            except Exception as e:
                results['gRPC'] = {
                    'status': 'unhealthy',
                    'error': str(e)
                }
        
        # GraphQL health check
        if 'graphql' in self.clients and self.clients['graphql']:
            try:
                # Simulate GraphQL health check
                results['GraphQL'] = {
                    'status': 'healthy',
                    'details': {'schema_loaded': True}
                }
            except Exception as e:
                results['GraphQL'] = {
                    'status': 'unhealthy',
                    'error': str(e)
                }
        
        return results
    
    def collect_metrics(self) -> Dict[str, Any]:
        """Collect API metrics."""
        metrics = {
            'timestamp': time.time(),
            'endpoints': {},
            'aggregated': {}
        }
        
        # Simulate metric collection
        metrics['endpoints'] = {
            'REST': {
                'requests_total': 1523,
                'requests_per_minute': 25.4,
                'avg_latency_ms': 112.3,
                'error_rate': 0.02
            },
            'gRPC': {
                'requests_total': 892,
                'requests_per_minute': 14.9,
                'avg_latency_ms': 87.6,
                'error_rate': 0.01
            },
            'GraphQL': {
                'queries_total': 456,
                'mutations_total': 234,
                'avg_latency_ms': 145.2,
                'error_rate': 0.03
            }
        }
        
        # Calculate aggregated metrics
        total_requests = sum(
            ep.get('requests_total', 0) + ep.get('queries_total', 0) + ep.get('mutations_total', 0)
            for ep in metrics['endpoints'].values()
        )
        
        metrics['aggregated'] = {
            'total_requests': total_requests,
            'avg_error_rate': np.mean([ep.get('error_rate', 0) for ep in metrics['endpoints'].values()]),
            'system_health': 'healthy' if all(
                ep.get('error_rate', 1) < 0.05 for ep in metrics['endpoints'].values()
            ) else 'degraded'
        }
        
        self.metrics.append(metrics)
        return metrics
    
    def generate_report(self) -> str:
        """Generate monitoring report."""
        latest_metrics = self.collect_metrics()
        health_status = self.health_check_all()
        
        report = []
        report.append("API Monitoring Report")
        report.append("=" * 50)
        
        # Health status
        report.append("\nHealth Status:")
        for api, status in health_status.items():
            report.append(f"  {api}: {status['status'].upper()}")
        
        # Performance metrics
        report.append("\nPerformance Metrics:")
        for api, metrics in latest_metrics['endpoints'].items():
            report.append(f"\n  {api}:")
            for metric, value in metrics.items():
                if isinstance(value, float):
                    report.append(f"    {metric}: {value:.2f}")
                else:
                    report.append(f"    {metric}: {value}")
        
        # System summary
        report.append("\nSystem Summary:")
        for metric, value in latest_metrics['aggregated'].items():
            if isinstance(value, float):
                report.append(f"  {metric}: {value:.2f}")
            else:
                report.append(f"  {metric}: {value}")
        
        return "\n".join(report)


# Create monitor
api_clients = {
    'rest': rest_client if 'rest_client' in locals() else None,
    'grpc': grpc_client if 'grpc_client' in locals() else None,
    'graphql': graphql_client if 'graphql_client' in locals() else None
}

monitor = APIMonitor(api_clients)

# Generate monitoring report
print(monitor.generate_report())

# Visualize metrics
metrics = monitor.collect_metrics()

# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# API request distribution
api_names = list(metrics['endpoints'].keys())
request_counts = [
    metrics['endpoints'][api].get('requests_total', 0) +
    metrics['endpoints'][api].get('queries_total', 0) +
    metrics['endpoints'][api].get('mutations_total', 0)
    for api in api_names
]

axes[0].bar(api_names, request_counts)
axes[0].set_xlabel('API Type')
axes[0].set_ylabel('Total Requests')
axes[0].set_title('Request Distribution by API')

# Latency comparison
latencies = [metrics['endpoints'][api].get('avg_latency_ms', 0) for api in api_names]

axes[1].bar(api_names, latencies, color='orange')
axes[1].set_xlabel('API Type')
axes[1].set_ylabel('Average Latency (ms)')
axes[1].set_title('Average Latency by API')

plt.tight_layout()
plt.show()

## 10. Best Practices Summary

In [ ]:
# API Usage Best Practices Summary
best_practices = {
    'Authentication': [
        'Always use secure API keys',
        'Implement token refresh mechanism',
        'Use OAuth 2.0 for production',
        'Never expose credentials in code'
    ],
    'Error Handling': [
        'Implement exponential backoff',
        'Use circuit breaker pattern',
        'Log all errors for debugging',
        'Provide meaningful error messages'
    ],
    'Performance': [
        'Batch requests when possible',
        'Implement request caching',
        'Use connection pooling',
        'Monitor latency and throughput'
    ],
    'Rate Limiting': [
        'Respect API rate limits',
        'Implement client-side throttling',
        'Use request queuing',
        'Monitor quota usage'
    ],
    'Protocol Selection': [
        'REST: Simple CRUD operations',
        'gRPC: High-performance, streaming',
        'GraphQL: Flexible queries, reduce over-fetching',
        'WebSocket: Real-time bidirectional communication'
    ]
}

print("API Usage Best Practices:")
print("="*50)

for category, practices in best_practices.items():
    print(f"\n{category}:")
    for practice in practices:
        print(f"  - {practice}")

# Performance comparison table
comparison_data = {
    'Protocol': ['REST', 'gRPC', 'GraphQL', 'WebSocket'],
    'Latency': ['Medium', 'Low', 'Medium-High', 'Very Low'],
    'Throughput': ['Medium', 'High', 'Medium', 'High'],
    'Complexity': ['Low', 'Medium', 'Medium-High', 'Medium'],
    'Use Case': ['CRUD', 'Microservices', 'Flexible Queries', 'Real-time']
}

comparison_df = pd.DataFrame(comparison_data)

print("\nProtocol Comparison:")
print("="*50)
print(comparison_df.to_string(index=False))

## 11. Conclusions and Next Steps

### API Usage Summary

This tutorial demonstrated comprehensive API usage patterns:

1. **REST API**: Simple HTTP-based communication with JSON
2. **gRPC**: High-performance RPC with Protocol Buffers
3. **GraphQL**: Flexible query language for precise data fetching
4. **WebSocket**: Real-time bidirectional communication
5. **Error Handling**: Exponential backoff, circuit breakers
6. **Rate Limiting**: Client-side throttling and quota management
7. **Performance Testing**: Latency, throughput, concurrent requests
8. **Monitoring**: Health checks, metrics collection, reporting

### Key Takeaways

1. **Protocol Selection**: Choose based on use case requirements
2. **Error Resilience**: Always implement retry logic and fallbacks
3. **Performance Optimization**: Batch requests and use caching
4. **Security First**: Secure authentication and data transmission
5. **Monitoring Essential**: Track health and performance metrics

### Next Steps

1. **Advanced Integration**:
   - Implement API gateway patterns
   - Use service mesh for microservices
   - Apply API versioning strategies

2. **Performance Optimization**:
   - Implement response caching
   - Use CDN for static content
   - Apply request deduplication

3. **Security Enhancement**:
   - Implement mTLS for gRPC
   - Use API key rotation
   - Apply request signing

4. **Production Deployment**:
   - Set up load balancing
   - Implement blue-green deployment
   - Configure auto-scaling

### References

For deeper understanding, consult:
- Service integration: `notebooks/tutorials/08_service_integration.ipynb`
- API documentation: `docs/api_reference/`
- Deployment guide: `docs/user_guide/deployment.md`
- Performance tuning: `docs/operations/runbooks/`